### Recurrent Neural Networks

This from a great little lesson by the folks at [wandb](https://www.youtube.com/watch?v=8lbGjKhrJOo).

RNNs are great for data with time as a factor (time series).

The specific data set we'll be working on is a small csv of daily minimum temperature in Melbourne over a period of 10 years.

In [19]:
# import packages
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import CuDNNLSTM, LSTM, SimpleRNN, Dropout
from keras.callbacks import LambdaCallback

from plotutil import PlotCallback

In [3]:
# import data
df = pd.read_csv('../data/daily-min-temperatures.csv')
data = df.Temp.astype('float32').values

We are going to use a commonly used technique called a sliding window for our time-series data. This basically involves looking at a window of data and calling it your features while using the succeeding column or period as the label. 

Terrible definition but it looks a little something like this: ![pic](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2018/11/Sliding-Window-Approach-to-Modeling-Time-Series.png)

I'll be using a lookback window of 20 here.

In [4]:
lookback_window = 20

In [5]:
# convert an array of values into a dataset matrix
def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset) - lookback_window-1):
        a = dataset[i:(i+lookback_window)]
        dataX.append(a)
        dataY.append(dataset[i + lookback_window])
    return np.array(dataX), np.array(dataY)

In [6]:
# split data into train/test sets
split_point = int(len(data) * 0.70)
train = data[:split_point]
test  = data[split_point:]

Above, notice we don't randomly split the data. Why? **Because we want to preserve the time aspect!!**

In [7]:
# create lookback matrices
trainX, trainY = create_dataset(train)
testX,  testY  = create_dataset(test)

In [8]:
# add a new dimension for compliance 
trainX = trainX[:, :, np.newaxis]
testX  = testX[:, :, np.newaxis]

In [ ]:
# create and fit the RNN
model = Sequential()
model.add(Flatten(input_shape=(lookback_window, 1)))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam', metrics=['mae']) # we use mean absolute error as our metric
model.fit(trainX, trainY, epochs=1000, batch_size=10,
          validation_data=(testX, testY))
          #,callbacks=[PlotCallback(trainX, trainY, testX, testY, lookback_window)])